# Part 1
Scraping of Toronto neighborhoods from the Wikipedia page (https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969).

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
data = requests.get(url).text
data

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"27ddcf54-302a-4bd6-81a9-00154555cd49","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1011037969,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ont

In [3]:
soup = BeautifulSoup(data,"html5lib")
tables = soup.find('table')
len(tables) # returns the number of tables found

2

In [4]:
for i, table in enumerate(tables):
    if "Borough" in str(table):
        table_index = i

print(table_index)

1


In [5]:
df = pd.read_html(str(tables), flavor='bs4')[0]
df.head()
#df.shape

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df_clean = df[df['Borough']!='Not assigned']
df_clean.head()
#df_clean.shape

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df_test = df_clean[df_clean['Neighbourhood'] == 'Not assigned']
df_test

#this result shows that there are no rows where Neighbourhood is 'Not Assigned', hence we can proceed to use df_clean

,Postal Code,Borough,Neighbourhood


In [8]:
col_names = ['PostalCode','Borough','Neighbourhood']
df_clean.columns = col_names
df_clean.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df_clean.shape

(103, 3)

# Part 2
Adding Lat and Log info for each neighbourhood

In [10]:
file = 'Geospatial_Coordinates.csv'
postcodes = pd.read_csv(file)
postcodes.head()
df_merged = df_clean.join(postcodes.set_index('Postal Code'), on = 'PostalCode')
df_merged.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
df_merged.shape

(103, 5)

# Part 3
Exploring and clustering the Toronto neighbourhoods

First, import all the dependencies

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Using geopy and folium, create a map of Toronto with neighbourhoods superimposed

In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="yyz_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [22]:
# trim to just the Toronto Borough

df_trim = df_merged[df_merged['Borough'].str.contains('Toronto')]
df_trim.head()
df_trim.shape

(40, 5)

In [23]:
# create map  using latitude and longitude values
map_yyz = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_trim['Latitude'], df_trim['Longitude'], df_trim['Borough'], df_trim['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_yyz)  
    
map_yyz

Define Foursquare credentials

In [18]:
CLIENT_ID = 'KTU3QFEMD0TJLOZHZUXBS3SSRMTTJAV3DDVJBTW4JF0STTSP' # your Foursquare ID
CLIENT_SECRET = 'VPMHRH4DYUP13WZLEIGSE21WHYOCLY3T0IZX43QWLADRSYC5' # your Foursquare Secret
VERSION = '20210320' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KTU3QFEMD0TJLOZHZUXBS3SSRMTTJAV3DDVJBTW4JF0STTSP
CLIENT_SECRET:VPMHRH4DYUP13WZLEIGSE21WHYOCLY3T0IZX43QWLADRSYC5


Trying out for the first Toronto neighbourhood

In [24]:
nhood = df_trim.loc[4, 'Neighbourhood']
print(nhood)
nhood_lat = df_trim.loc[4, 'Latitude']
print("Lat:",nhood_lat)
nhood_long = df_trim.loc[4, 'Longitude']
print("Long:",nhood_long)

Regent Park, Harbourfront
Lat: 43.6542599
Long: -79.3606359


In [25]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, nhood_lat, nhood_long, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=KTU3QFEMD0TJLOZHZUXBS3SSRMTTJAV3DDVJBTW4JF0STTSP&client_secret=VPMHRH4DYUP13WZLEIGSE21WHYOCLY3T0IZX43QWLADRSYC5&ll=43.6542599,-79.3606359&v=20210320&radius=500&limit=100'

In [26]:
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-26-3c8d9b57d873>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


Creating a function for all neighbourhoods in Toronto

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
yyz_venues = getNearbyVenues(df_trim['Neighbourhood'],df_trim['Latitude'],df_trim['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

In [30]:
yyz_venues.head()
#yyz_venues.groupby('Neighborhood').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [31]:
len(yyz_venues['Venue Category'].unique())

234

Analyzing the categories of venues for each neighbourhood

In [32]:
# one hot encoding
yyz_onehot = pd.get_dummies(yyz_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
yyz_onehot['Nhood'] = yyz_venues['Neighborhood'] 
#yyz_onehot.head()

# move neighborhood column to the first column
fixed_columns = [yyz_onehot.columns[-1]] + list(yyz_onehot.columns[:-1])
yyz_onehot = yyz_onehot[fixed_columns]

#yyz_onehot.head()
yyz_onehot.shape

(1613, 235)

In [33]:
yyz_grouped = yyz_onehot.groupby('Nhood').mean().reset_index()
yyz_grouped

,Nhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.016667,0.050000,0.000000,0.000000,0.00,0.016667,0.016667,0.000000,0.033333,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.016667,0.050000,0.083333,0.000000,0.000000,0.000000,0.00,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000

In [34]:
yyz_grouped.shape

(40, 235)

Sort the venues in descending order of common-ness

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe displaying the top 5 venues for each neighbourhood

In [38]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = yyz_grouped['Nhood']

for ind in np.arange(yyz_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yyz_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.shape

(40, 6)

Cluster the 40 neighbourhoods into 4 clusters

In [40]:
# set number of clusters
kclusters = 4

yyz_grouped_clustering = yyz_grouped.drop('Nhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yyz_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 1, 2,
       2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2])

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,2,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market
1,2,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio
2,2,"Business reply mail Processing Centre, South C...",Pizza Place,Garden,Restaurant,Butcher,Burrito Place
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar
4,2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant
5,0,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store
6,2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar
7,2,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym
8,2,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Café
9,2,Davisville North,Gym / Fitness Center,Hotel,Park,Department Store,Sandwich Place


In [43]:
#duplicate the df_trim
yyz_data = df_trim
yyz_data.shape

(40, 5)

In [44]:
# merge the yyz_data which has lat,long,nhood names with the neighborhoods_venues_sorted data with the cluster label
yyz_joined = yyz_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

yyz_joined.head() 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Pub,Restaurant
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Diner,Park,Bar
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Japanese Restaurant
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cosmetics Shop,Gastropub,Cocktail Bar
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Pub,Health Food Store,Trail,Distribution Center


Visualise results

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(yyz_joined['Latitude'], yyz_joined['Longitude'], yyz_joined['Neighbourhood'], yyz_joined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The above map shows the four clusters in the four different colours. Let's take a look at the characteristics of the individual clusters

In [47]:
### Cluster 1
yyz_joined.loc[yyz_joined['Cluster Labels'] == 0, yyz_joined.columns[[2] + list(range(5, yyz_joined.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
30,The Beaches,0,Neighborhood,Pub,Health Food Store,Trail,Distribution Center
41,Christie,0,Grocery Store,Café,Park,Coffee Shop,Candy Store
94,Roselawn,0,Health & Beauty Service,Garden,Farmers Market,Falafel Restaurant,Event Space
95,"Runnymede, The Junction, Weston-Pellam Park, C...",0,Grocery Store,Convenience Store,Brewery,Yoga Studio,Distribution Center


In [48]:
### Cluster 2
yyz_joined.loc[yyz_joined['Cluster Labels'] == 1, yyz_joined.columns[[2] + list(range(5, yyz_joined.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
103,"Forest Hill North & West, Forest Hill Road Park",1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio
129,"Moore Park, Summerhill East",1,Park,Lawyer,Trail,Summer Camp,Yoga Studio
147,Rosedale,1,Park,Playground,Trail,Yoga Studio,Diner


In [49]:
### Cluster 3
yyz_joined.loc[yyz_joined['Cluster Labels'] == 2, yyz_joined.columns[[2] + list(range(5, yyz_joined.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,"Regent Park, Harbourfront",2,Coffee Shop,Bakery,Park,Pub,Restaurant
6,"Queen's Park, Ontario Provincial Government",2,Coffee Shop,Sushi Restaurant,Diner,Park,Bar
13,"Garden District, Ryerson",2,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Japanese Restaurant
22,St. James Town,2,Coffee Shop,Café,Cosmetics Shop,Gastropub,Cocktail Bar
31,Berczy Park,2,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market
40,Central Bay Street,2,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant
49,"Richmond, Adelaide, King",2,Coffee Shop,Café,Clothing Store,Restaurant,Deli / Bodega
50,"Dufferin, Dovercourt Village",2,Pharmacy,Bakery,Park,Pool,Café
58,"Harbourfront East, Union Station, Toronto Islands",2,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant
59,"Little Portugal, Trinity",2,Bar,Men's Store,Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant


In [50]:
### Cluster 4
yyz_joined.loc[yyz_joined['Cluster Labels'] == 3, yyz_joined.columns[[2] + list(range(5, yyz_joined.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
93,Lawrence Park,3,Park,Bus Line,Swim School,Yoga Studio,Distribution Center


### Cluster 1 appears tp be nhoods with some services. They are located on the outskirts of Toronto.

### Cluster 2 appears to be nhoods with nature areas.

### Cluster 3 appears to be nhoods with cafes, coffee shops etc. They are located closest to Downtown Toronto.

### Cluster 4 is less clear since there's only 1 element in the cluster. 